<a href="https://colab.research.google.com/github/LeninGF/CoursesNotes/blob/main/InteligenciaArtificalGenerativa/Problems/LSTM-GRU/EjerciciosLSTM-GRU-IAG-2024B_LeninFalconi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios sobre Redes Neuronales LSTM y GRU

Coder: Lenin G. Falconí

Asignatura: Tópicos Especiales (Inteligencia Artificial)

Fecha: 2024-11-06

## Problema 1 Comparar One-Hot y Embedding
Analice la representación one-hot y la de embedding. Inicializar las
dependencias necesarias y adaptar el código para usarlo con IMDb Dataset de Movie Reviews.  

In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, GRU, LSTM

# Parameters
vocab_size = 8000 # 8000
max_text_len = 200
word_vec_dim =100

print(tf.__version__)

2.17.0


Cargando los datos del IMDB dataset

In [9]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train.shape, x_test.shape

((25000,), (25000,))

Pad sequences

In [10]:
x_train = pad_sequences(x_train, maxlen=max_text_len)
x_test = pad_sequences(x_test, maxlen=max_text_len)

Crear un dataset de tipo tf.data

In [11]:
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

One-hot encoding the sequences

In [6]:
# x_train = tf.keras.utils.to_categorical(x_train, num_classes=vocab_size)
# x_test = tf.keras.utils.to_categorical(x_test, num_classes=vocab_size)
# x_train.shape, x_test.shape

((25000, 200, 100), (25000, 200, 100))

In [7]:
# x_train[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# Apply one-hot encoding within the dataset pipeline
def one_hot_encode(text, label):
    encoded_text = tf.keras.utils.to_categorical(text, num_classes=vocab_size)
    return encoded_text, label

train_ds = train_ds.map(one_hot_encode)
test_ds = test_ds.map(one_hot_encode)

In [18]:
for x, y in train_ds.take(5):
    print(x.shape, y.shape)

(32, 200, 8000) (32,)
(32, 200, 8000) (32,)
(32, 200, 8000) (32,)
(32, 200, 8000) (32,)
(32, 200, 8000) (32,)


In [22]:
import numpy as np
t = np.array([
    [
        [1,2,3], [1,2,3], [1,2,3]
        ],
         [
             [4,5,6],[4,5,6],[4,5,6]
             ]
    ])
t.shape

(2, 3, 3)

Crear Modelo One-Hot

In [5]:
# Build the model
model_one_hot = Sequential([
    Flatten(input_shape=(max_text_len, vocab_size)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
], name='model_one_hot')

model_one_hot.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "model_one_hot"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       2,560,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,560,257 (9.77 MB)

 Trainable params: 2,560,257 (9.77 MB)

 Non-trainable params: 0 (0.00 B)